<p style="font-size:300%; text-align:center"> Telco Customer Churn modeling</p>
<p style="font-size:150%; text-align:center"> Focused customer retention programs <br> MOD3 Project - 2. Scrub</p>


In [1]:
# import important libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# import important and explore dataset
df = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.sample(6).T

,1889,3878,3709,4553,273,2535
customerID,8069-YQQAJ,2223-GDSHL,2103-ZRXFN,5914-XRFQB,4716-HHKQH,2000-DHJUY
gender,Male,Male,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,1,1
Partner,No,Yes,No,Yes,Yes,Yes
Dependents,No,Yes,No,No,No,No
tenure,14,71,1,72,20,70
PhoneService,No,Yes,Yes,Yes,Yes,Yes
MultipleLines,No phone service,Yes,Yes,Yes,Yes,Yes
InternetService,DSL,Fiber optic,DSL,Fiber optic,Fiber optic,DSL
OnlineSecurity,No,Yes,No,Yes,No,Yes


## Any obvious features to eliminate? 
customerID needs to be removed for data explorationa, analysis and model building. 

In [3]:
# let's remove customer ID as it's not helpful fro data analysis and prediction
df = df.drop(['customerID'], axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 1.1+ MB


First look at the data types show TotalCharge should be a numerical value. Let's first explore this feature. 

In [5]:
df.TotalCharges.value_counts()

20.2       11
           11
19.75       9
19.9        8
19.65       8
           ..
5815.15     1
2011.4      1
5655.45     1
655.5       1
125.95      1
Name: TotalCharges, Length: 6531, dtype: int64

In [6]:
# there appear to be 11 empty values. Let's replace them with nan. and since it's less than .2% of the data we can drop them
df.TotalCharges = df.TotalCharges.replace(" ",np.nan)
df.dropna(inplace=True)

In [7]:
for col in df.columns:
    display(df[col].value_counts())

Male      3549
Female    3483
Name: gender, dtype: int64

0    5890
1    1142
Name: SeniorCitizen, dtype: int64

No     3639
Yes    3393
Name: Partner, dtype: int64

No     4933
Yes    2099
Name: Dependents, dtype: int64

1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: tenure, Length: 72, dtype: int64

...

Yes    4168
No     2864
Name: PaperlessBilling, dtype: int64

Electronic check             2365
Mailed check                 1604
Bank transfer (automatic)    1542
Credit card (automatic)      1521
Name: PaymentMethod, dtype: int64

20.05     61
19.85     44
19.90     44
19.95     44
19.65     43
          ..
92.35      1
35.60      1
72.85      1
67.70      1
113.30     1
Name: MonthlyCharges, Length: 1584, dtype: int64

20.2       11
19.75       9
19.65       8
20.05       8
19.9        8
           ..
5815.15     1
2011.4      1
5655.45     1
655.5       1
125.95      1
Name: TotalCharges, Length: 6530, dtype: int64

No     5163
Yes    1869
Name: Churn, dtype: int64

In [8]:
# Yes/No maybe easier for the EDA and iNterpret
df.to_csv("data/telco_clean_1.csv", index=False)

## removing redundancies

* "No phone service" of "MultipleLines" means no multiple lines by definition. Thus, we can change "no phone service" to simply "no". <br>
* Similarly "No internet service" in features "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV" and "StreamingMovies" can be changed to "no" as well 

In [9]:
df.replace(to_replace=["No phone service", "No internet service", "Fiber optic"], 
           value = ["No_phone", "No_internet", "Fiber_optic"], inplace=True)

In [10]:
# Make the target and Yes/No categorical features numerical
cols=['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in cols:
    df[col].replace(to_replace=["Yes", "No"], value = [1, 0], inplace=True)

In [11]:
# clean the 'Contract'
df['Contract'] = df['Contract'].apply(lambda x: x.strip().replace(" ","_").replace("-","_"))
# clean the PaymentMethod
df['PaymentMethod'] = df['PaymentMethod'].apply(lambda x: x.replace("automatic", "").replace("(", "").replace(")", "").strip())
df['PaymentMethod'] = df['PaymentMethod'].apply(lambda x: x.strip().replace(" ","_").replace("\(automatic\)", ""))
#display(df['PaymentMethod'].value_counts())

In [12]:
for col in df.columns:
    display(df[col].value_counts())

Male      3549
Female    3483
Name: gender, dtype: int64

0    5890
1    1142
Name: SeniorCitizen, dtype: int64

0    3639
1    3393
Name: Partner, dtype: int64

0    4933
1    2099
Name: Dependents, dtype: int64

1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: tenure, Length: 72, dtype: int64

...

1    4168
0    2864
Name: PaperlessBilling, dtype: int64

Electronic_check    2365
Mailed_check        1604
Bank_transfer       1542
Credit_card         1521
Name: PaymentMethod, dtype: int64

20.05     61
19.85     44
19.90     44
19.95     44
19.65     43
          ..
92.35      1
35.60      1
72.85      1
67.70      1
113.30     1
Name: MonthlyCharges, Length: 1584, dtype: int64

20.2       11
19.75       9
19.65       8
20.05       8
19.9        8
           ..
5815.15     1
2011.4      1
5655.45     1
655.5       1
125.95      1
Name: TotalCharges, Length: 6530, dtype: int64

0    5163
1    1869
Name: Churn, dtype: int64

In [13]:
df.sample(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4001,Male,0,1,1,45,1,Yes,Fiber_optic,No,Yes,Yes,Yes,Yes,Yes,Two_year,1,Bank_transfer,109.75,4900.65,0
5,Female,0,0,0,8,1,Yes,Fiber_optic,No,No,Yes,No,Yes,Yes,Month_to_month,1,Electronic_check,99.65,820.5,1
3482,Male,0,0,0,19,1,No,DSL,No,No,No,No,No,No,Month_to_month,1,Bank_transfer,45.00,865.85,0
2256,Female,0,0,1,15,1,No,DSL,Yes,Yes,Yes,Yes,No,No,One_year,0,Credit_card,65.60,1010,0
2394,Male,0,0,0,71,1,Yes,Fiber_optic,No,Yes,No,No,No,No,One_year,1,Credit_card,79.05,5552.5,0
3556,Male,0,1,1,6,1,No,No,No_internet,No_internet,No_internet,No_internet,No_internet,No_internet,One_year,0,Mailed_check,19.45,113.5,0
1857,Female,0,1,0,65,1,No,DSL,Yes,No,Yes,Yes,No,No,Two_year,0,Credit_card,58.90,3857.1,0
6895,Male,0,1,1,54,0,No_phone,DSL,No,No,Yes,Yes,No,Yes,Two_year,0,Credit_card,46.00,2424.05,0
9,Male,0,0,1,62,1,No,DSL,Yes,Yes,No,No,No,No,One_year,0,Bank_transfer,56.15,3487.95,0
3489,Female,0,1,0,26,1,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,One_year,1,Credit_card,90.10,2312.55,0


## Scrubbing done
save file 


In [14]:
df.to_csv("data/telco_clean.csv", index=False)